# Import Libraries

In [3]:
import pandas as pd
import datetime as datetime
import igraph

In [9]:
df = pd.read_csv('/home/poulami/Documents/Github/Peer-Review-Analysis/data/personreview.csv')  #Enter path of the file personreview.csv
#person = pd.read_csv('/home/poulami/Documents/Github/Peer-Review-Analysis/data/person.csv')
review = pd.read_csv('/home/poulami/Documents/Github/Peer-Review-Analysis/data/result.csv')

# Generate network
### vertices  -  persons
### edges - no of common reviews commented on

In [13]:
def generatenetwork (start, end, index):
    
    
    df1 = df
    df1['date'] = df['date'].apply(pd.to_datetime, errors='coerce')      #typecasts the column date to type datetime
    #print(df1.dtypes)
    df1=df1.sort_values(by='date')                                        #Sorts the dataframe in ascending order of dates
    #print(df1)
    #start = input("Enter start date (yyyymmdd):")
    #end = input("Enter end date (yyyymmdd):")
    s = datetime.datetime.strptime(str(start), '%Y-%m-%d %H:%M:%S.%f')
    e = datetime.datetime.strptime(str(end) , '%Y-%m-%d %H:%M:%S.%f')
    df1 = df1.set_index(['date'])                                         #Required for the next step.  Sets date as an index
    df1 = df1.loc[s:e]                                                    #Takes only the comments made between dates s and e
    df1 = df1.reset_index(drop=True)                                      #Drops the index column 'date' as it is not needed anymore  
    df1 = df1.drop_duplicates(subset=['sender', 'issue'])                 #Removes duplicate columns
    print('top')
    print(df1.shape[0])
    
    ##################################################################################
    #    Compares the data frames w.r.t the set keys and extracts selected rows      #
    ##################################################################################

    person = df1.drop_duplicates(['sender'])
    review = df1.drop_duplicates(['issue'])
    keys = ['sender']
    i1 = df1.set_index(keys).index
    i2 = person.set_index(keys).index
    df1 = df1[i1.isin(i2)]
    keys = ['issue']
    i1 = df1.set_index(keys).index
    i2 = review.set_index(keys).index
    df1 = df1[i1.isin(i2)]
    #print(df1)
    ##################################################################################
    #                          Initialize the graph                                  #
    ##################################################################################
    g = Graph.Full(0)
    print(person.shape[0])
    for i in (person['sender']):
        g.add_vertices([i])
        #print(g)
    g.es["weight"]=0
    ##################################################################################
    #                    Creating edges and assigning edge weights                   #
    ##################################################################################
    df1 = df1[df1.duplicated(subset=['issue'] ,keep=False)]               #Keeps only the the reviews which occur twice. This is done because 
    #print('edgecre')                                                            
    sender = df1[['sender']]                                                                                                                                               #
    #print(sender)              2008-09-02 20:13:34.526552 2008-10-03 05:28:56.748409                                                                                                                                            #  CAN BE IMPROVED
    sender = sender.drop_duplicates(subset=['sender'],keep= 'first' )     #Gets unique sender names along with issues (The  dataframe sender is used for looping through)  #
    print('1')                         
    for i in (sender['sender']):   
        set1=df1[df1['sender'].isin([i])]                                 #Create first set                                                           
        sender = sender.drop(sender.index[0])                             #Deletes the first element of sender   
        for j in (sender['sender']):                                      #Creates second set               
            set2=df1[df1['sender'].isin([j])]
            newset = pd.concat([set1, set2])                              #set union
            #print(newset)
            newset = newset[newset.duplicated(subset=['issue'] ,keep='first')]  #Set intersection
            weight = (newset.shape[0])
            if(weight != 0):
                g[i,j] = weight
    g.vs['id'] = g.vs['name']                                             #assigns vertex id
    g.write_pajek(("T" + str(index+1) + "-nw.net"))                                             #writes to pajek file
#



In [18]:
generatenetwork('2008-09-02 20:13:34.526552', '2008-10-03 05:28:56.748409',1 )

# Generate network graph for each time stamp

In [16]:
head = datetime.datetime.strptime('2008-09-02 20:13:34.526552', '%Y-%m-%d %H:%M:%S.%f') 
tail = datetime.datetime.strptime('2012-10-31 03:02:05.619400', '%Y-%m-%d %H:%M:%S.%f') 
print(head,'\n',tail)
step = (tail-head)/50
print(step)
sum=0
for i in range(0,50):
        start  = head+i*step
        end = head +(1+i)*step
        print(i,start, head, end)
        generatenetwork(head, end,i)


2008-09-02 20:13:34.526552 
 2012-10-31 03:02:05.619400
30 days, 9:15:22.221857
0 2008-09-02 20:13:34.526552 2008-09-02 20:13:34.526552 2008-10-03 05:28:56.748409
1 2008-10-03 05:28:56.748409 2008-09-02 20:13:34.526552 2008-11-02 14:44:18.970266
2 2008-11-02 14:44:18.970266 2008-09-02 20:13:34.526552 2008-12-02 23:59:41.192123
3 2008-12-02 23:59:41.192123 2008-09-02 20:13:34.526552 2009-01-02 09:15:03.413980
4 2009-01-02 09:15:03.413980 2008-09-02 20:13:34.526552 2009-02-01 18:30:25.635837
5 2009-02-01 18:30:25.635837 2008-09-02 20:13:34.526552 2009-03-04 03:45:47.857694
6 2009-03-04 03:45:47.857694 2008-09-02 20:13:34.526552 2009-04-03 13:01:10.079551
7 2009-04-03 13:01:10.079551 2008-09-02 20:13:34.526552 2009-05-03 22:16:32.301408
8 2009-05-03 22:16:32.301408 2008-09-02 20:13:34.526552 2009-06-03 07:31:54.523265
9 2009-06-03 07:31:54.523265 2008-09-02 20:13:34.526552 2009-07-03 16:47:16.745122
10 2009-07-03 16:47:16.745122 2008-09-02 20:13:34.526552 2009-08-03 02:02:38.966979
11 200